<a href="https://colab.research.google.com/github/daveselinger/covid-19-hackathon/blob/master/by_state_covid_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Purpose**

The purpose of this analysis is to help people understand the severity of the COVID19 in a more realistic fashion.

We, the authors, believe that the analysis of COVID19 using national data is lacking in precision and actionability. Further, we feel that it may not be really representative of the "on-the-ground" experience: i.e., you can't average the experience of rural Montana with that of metropolitan Manhattan; or in fact even compare the metropolitan area of Seattle with that in San Diego.

**Open Source**

This code, its methodology, and conclusions are all released without any additional warranties or expectations using the MIT Open Source License.

**Social implications and responsibility**

While we are statisticians we recognize that there may be conclusions drawn from this analysis that have political and or personal ramifications for people. We therefore release this information to the open source with the best of intentions and with the hopes that these conclusions are used to save lives in a period of intense anxiety and volatility. In a world when it is hard to know "which facts" to believe, we have done everything we can to remove bias, to be explicit and transparent in the assumptions made and to be clear of our intent: We believe that more data will be better; we are disappointed in the lack of testing data -- and that is clear from our conclusions (these conslusions must be caveated with the fact that our model is affected by the variance because of the CDC's inability to test at the scale needed to face this problem); we believe that even with this volatility of data in a time of such dire consequences as has been proven in Wuhan China and Northern Italy, to not reach the best possible conclusions based on the data that we do have would be a dereliction of our duty and an act of ignorance. Therefore we submit this analysis and conclusions with nothing less than the best hopes for our citizens and with the desire for our local and state governments to do what is necessary regardless of any resistance they may face: We implore state health departments to act faster--even faster than the CDC and FDA might desire.


In [1]:
!wget https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv
!pip install -U plotly
!pip install plotly-geo
!pip install geopandas==0.3.0
!pip install pyshp==1.2.10
!pip install chart-studio


--2020-03-19 15:33:53--  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74283 (73K) [text/plain]
Saving to: ‘time_series_19-covid-Confirmed.csv’

time_series_19-covi 100%[===================>]  72.54K  --.-KB/s    in 0.02s   

2020-03-19 15:33:58 (2.94 MB/s) - ‘time_series_19-covid-Confirmed.csv’ saved [74283/74283]

     |████████████████████████████████| 7.1MB 2.8MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 23.7MB 1.2MB/s 
     |████████████████████████████████| 890kB 2.8MB/s 
     |█████████████████

In [2]:
from __future__ import print_function

import numpy as np 
import chart_studio.plotly as py
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import datetime
import math
import scipy

pd.__version__


'0.25.3'

In [3]:
covid_across_countries_df = pd.read_csv('time_series_19-covid-Confirmed.csv')
global_timeseries = covid_across_countries_df.set_index(['Country/Region'])
global_timeseries.head(10)


,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Thailand,NaN,15.0000,101.0000,2,3,5,7,8,8,14,14,14,19,19,19,19,25,25,25,25,32,32,32,33,33,33,33,33,34,35,35,35,35,35,35,35,35,37,40,40,41,42,42,43,43,43,47,48,50,50,50,53,59,70,75,82,114,147,177,212
Japan,NaN,36.0000,138.0000,2,1,2,2,4,4,7,7,11,15,20,20,20,22,22,45,25,25,26,26,26,28,28,29,43,59,66,74,84,94,105,122,147,159,170,189,214,228,241,256,274,293,331,360,420,461,502,511,581,639,639,701,773,839,825,878,889
Singapore,NaN,1.2833,103.8333,0,1,3,3,4,5,7,7,10,13,16,18,18,24,28,28,30,33,40,45,47,50,58,67,72,75,77,81,84,84,85,85,89,89,91,93,93,93,102,106,108,110,110,117,130,138,150,150,160,178,178,200,212,226,243,266,313
Nepal,NaN,28.1667,84.2500,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Malaysia,NaN,2.5000,112.5000,0,0,0,3,4,4,4,7,8,8,8,8,8,10,12,12,12,16,16,18,18,18,19,19,22,22,22,22,22,22,22,22,22,22,22,22,23,23,25,29,29,36,50,50,83,93,99,117,129,149,149,197,238,428,566,673,790
Canada,British Columbia,49.2827,-123.1207,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,4,4,4,4,4,4,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8,8,9,12,13,21,21,27,32,32,39,46,64,64,73,103,103,186
Australia,New South Wales,-33.8688,151.2093,0,0,0,0,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,6,6,13,22,22,26,28,38,48,55,65,65,92,112,134,171,210,267
Australia,Victoria,-37.8136,144.9631,0,0,0,0,1,1,1,1,2,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,7,7,9,9,10,10,10,11,11,15,18,21,21,36,49,57,71,94,121
Australia,Queensland,-28.0167,153.4000,0,0,0,0,0,0,0,1,3,2,3,2,2,3,3,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,9,9,9,11,11,13,13,13,15,15,18,20,20,35,46,61,68,78,94


In [4]:
us_timeseries = global_timeseries.loc[['US']]
us_timeseries.head(5)

,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
US,Washington,47.4009,-121.4905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,267,366,442,568,572,643,904,1076,1014
US,New York,42.1657,-74.9481,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,173,220,328,421,525,732,967,1706,2495
US,California,36.1162,-119.6816,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,177,221,282,340,426,557,698,751
US,Massachusetts,42.2302,-71.5301,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92,95,108,123,138,164,197,218,218
US,Diamond Princess,35.4437,139.6380,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,36,42,42,44,44,44,45,45,45,45,45,45,45,45,46,46,46,46,46,46,47,47,47


Now that we have the US data isloated, we can input this into a spatio-temporal map: Space represented as a map, and temporal as an animation.


In [5]:
!wget http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?#
state_populations = pd.read_csv('nst-est2019-alldata.csv?', index_col='NAME')
state_populations.head(10)



--2020-03-19 15:34:50--  http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?
Resolving www2.census.gov (www2.census.gov)... 184.25.191.212, 2600:1402:3800:297::208c, 2600:1402:3800:29f::208c, ...
Connecting to www2.census.gov (www2.census.gov)|184.25.191.212|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?&sec_ak_reference=18.246019b8.1584632091.7b0fc333 [following]
--2020-03-19 15:34:51--  https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?&sec_ak_reference=18.246019b8.1584632091.7b0fc333
Connecting to www2.census.gov (www2.census.gov)|184.25.191.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘nst-est2019-alldata.csv?’

nst-est2019-alldata     [ <=>                ]

,SUMLEV,REGION,DIVISION,STATE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,NPOPCHG_2016,NPOPCHG_2017,NPOPCHG_2018,NPOPCHG_2019,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,BIRTHS2016,BIRTHS2017,BIRTHS2018,BIRTHS2019,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,...,RDEATH2016,RDEATH2017,RDEATH2018,RDEATH2019,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RNATURALINC2016,RNATURALINC2017,RNATURALINC2018,RNATURALINC2019,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RINTERNATIONALMIG2016,RINTERNATIONALMIG2017,RINTERNATIONALMIG2018,RINTERNATIONALMIG2019,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RDOMESTICMIG2016,RDOMESTICMIG2017,RDOMESTICMIG2018,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
United States,10,0,0,0,308745538,308758105,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523,563561,2235208,2274116,2162725,2307293,2334155,2306148,2044228,1701962,1552022,987836,3973485,3936976,3940576,3963195,3992376,3962654,3901982,3824521,3791712,598691,2512442,2501531,2608019,...,8.400602,8.606413,8.668095,8.657569,4.706373,4.590575,4.231517,4.353645,4.045944,3.913875,3.438101,3.069450,2.921467,2.493773,2.682083,2.636187,2.921500,3.260435,3.252788,2.871957,2.153911,1.818059,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.493773,2.682083,2.636187,2.921500,3.260435,3.252788,2.871957,2.153911,1.818059
Northeast Region,20,1,0,0,55317240,55318443,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803,61691,224089,170993,126590,104205,28673,7646,16910,-12620,-63817,163466,646249,637860,635751,632433,634504,628030,618490,610713,602740,110878,470811,460915,479963,...,8.574497,8.883355,8.975017,9.025986,3.161491,3.177337,2.789974,2.898189,2.593611,2.632618,2.151103,1.920274,1.734402,3.680555,3.681434,3.428960,3.931182,4.009115,4.177449,3.809171,3.195141,2.394817,-2.792646,-3.719789,-3.898743,-4.917279,-6.071080,-6.667933,-5.646219,-5.329588,-5.254530,0.887909,-0.038355,-0.469783,-0.986097,-2.061965,-2.490484,-1.837048,-2.134447,-2.859713
Midwest Region,20,2,0,0,66927001,66929725,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004,44691,183384,178943,223636,184788,115416,126957,139241,109847,92376,212570,834866,830701,830890,836538,838012,831641,818785,801587,792343,140862,586660,584671,605165,...,9.115871,9.409840,9.209377,9.121680,3.700916,3.658587,3.346624,3.542205,3.125649,3.127816,2.621003,2.547252,2.482162,1.713906,1.767566,1.845688,1.951908,2.070856,2.106735,1.825671,1.380898,1.254708,-2.677835,-2.741509,-1.852612,-2.714878,-3.459293,-3.348519,-2.383041,-2.303653,-2.365881,-0.963930,-0.973943,-0.006924,-0.762969,-1.388437,-1.241784,-0.557370,-0.922755,-1.111173
South Region,20,3,0,0,114555744,114563030,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448,303650,1139842,1234686,1123192,1259637,1373304,1354419,1190429,1027244,1011015,368759,1509634,1504955,1504774,1525313,1545679,1541412,1519344,1494950,1481244,228435,963051,960791,1011227,...,8.695993,8.884440,9.003440,8.971661,4.734919,4.665975,4.189603,4.363271,4.101489,3.972326,3.473278,3.047185,2.871191,2.422447,2.898420,2.745376,3.013508,3.507594,3.521386,3.220112,2.415429,1.942372,2.708067,2.952037,2.546697,3.147993,3.769765,3.628688,2.978056,2.810090,3.261349,5.130513,5.850458,5.292073,6.161501,7.277358,7.150074,6.198168,5.225519,5.203720
West Regio

Get append data including population, hospitals, hospital beds.

In [6]:
#Need these abbreviations because Plotly requires 2-letter state abbreviations.

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# filter out some of the states in the state_populations like the Diamond Princess
good_states = state_populations.index.intersection(us_state_abbrev.keys())

us_state_pop = {}
for state_name in good_states:
  state_row = state_populations.loc[state_name]
  us_state_pop[state_name]=state_row["POPESTIMATE2019"]

print (us_state_pop)

{'Alabama': 4903185, 'Alaska': 731545, 'Arizona': 7278717, 'Arkansas': 3017804, 'California': 39512223, 'Colorado': 5758736, 'Connecticut': 3565287, 'Delaware': 973764, 'District of Columbia': 705749, 'Florida': 21477737, 'Georgia': 10617423, 'Hawaii': 1415872, 'Idaho': 1787065, 'Illinois': 12671821, 'Indiana': 6732219, 'Iowa': 3155070, 'Kansas': 2913314, 'Kentucky': 4467673, 'Louisiana': 4648794, 'Maine': 1344212, 'Maryland': 6045680, 'Massachusetts': 6892503, 'Michigan': 9986857, 'Minnesota': 5639632, 'Mississippi': 2976149, 'Missouri': 6137428, 'Montana': 1068778, 'Nebraska': 1934408, 'Nevada': 3080156, 'New Hampshire': 1359711, 'New Jersey': 8882190, 'New Mexico': 2096829, 'New York': 19453561, 'North Carolina': 10488084, 'North Dakota': 762062, 'Ohio': 11689100, 'Oklahoma': 3956971, 'Oregon': 4217737, 'Pennsylvania': 12801989, 'Rhode Island': 1059361, 'South Carolina': 5148714, 'South Dakota': 884659, 'Tennessee': 6829174, 'Texas': 28995881, 'Utah': 3205958, 'Vermont': 623989, 'Vi

**Hospitals**

Next we analyze hospitals. We use data from [ArcGIS OpenData](https://hifld-geoplatform.opendata.arcgis.com/datasets/hospitals/data) but for some reason I couldn't download this using a link, so I pulled it into Github and just stored it there.

**Assumptions**

These data have 2-3 key assumptions made in the modeling:
1. We assume that the only beds that can be used for ICU are in the "General Acute Care" class of hospital.
2. We assume that ICU beds comprise approximately 15%. We did not find a data set containing ICU details per state. This was based on an article from the NIH here: [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5520980/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5520980/)

In [7]:
hospital_beds_df = pd.read_csv('https://raw.githubusercontent.com/daveselinger/covid-19-hackathon/master/Hospitals.csv')
hospital_beds_df.replace(to_replace=-999, value=1, inplace=True)
print (hospital_beds_df.set_index('STATE').loc['AK'].head(5))
print (hospital_beds_df.groupby('TYPE').sum())

hospital_beds_df.index.value_counts()

hospital_bed_count_df = hospital_beds_df.where(hospital_beds_df['TYPE'] == "GENERAL ACUTE CARE").groupby('STATE')['BEDS'].sum()

                X          Y  OBJECTID  ...  BEDS    TRAUMA HELIPAD
STATE                                   ...                        
AK    -151.077827  60.493169       635  ...   108  LEVEL II       Y
AK    -149.800293  61.182950       636  ...   148  LEVEL II       N
AK    -131.685788  55.353719       637  ...    54  LEVEL IV       Y
AK    -147.739393  64.831745       638  ...   217  LEVEL IV       Y
AK    -165.405315  64.501918       639  ...    36  LEVEL IV       N

[5 rows x 33 columns]
                                X              Y  ...  TTL_STAFF    BEDS
TYPE                                              ...                   
CHILDREN            -11434.801713    4568.912875  ...        124   16853
CHRONIC DISEASE       -660.310220     369.804715  ...          9    1414
CRITICAL ACCESS     -99135.887396   41965.188956  ...       1031   27440
GENERAL ACUTE CARE -415767.410650  167018.239174  ...       4524  831577
LONG TERM CARE      -39202.381094   15554.476288  ...        43

**Computing State-by-State Statistics**

Some key assumptions in computing these data:
- 40% of cases will need hospitalization
- 15% of cases will need an ICU
- ICU's are 15% of total beds in a hospital
- 90% of the cases are in the key metropolitan area of the state (which is where the outbreak will occur in a state most likely due to virality/spread -- this is NOT ACTUALLY the case if we had metro/county level data which would be much more accurate)
- 40% of the population (and importantly here hospital beds/icu beds) are in this metropolitan areas

**Key assumptions and the desire to repeat this analysis with county-level data**

These last 2 assumptions are particularly important as the population-density of a state is not flat across its population. The most likely places for broadly spreading COVID are in places where many people are interacting frequently (i.e., metropolitan areas). Therefore, we have made the assumption that the case-incident data would reflect this distribution of cases. We chose 90% as an assumption based on the virulence and high R0 of this disease--the higher this value, the more exaggerated the impact of even a very few cases in a dense metro area. Furthermore, we estimate that only 40% of the population of a state lives in this based on an approximation from the WAPO combining "Metropolitan" and "Big City"
https://www.washingtonpost.com/business/2018/12/18/americans-say-theres-not-much-appeal-big-city-living-why-do-so-many-us-live-there/ (which indicates 12% big city, 21% big city suburb)

We recognize that these assumptions would be far better handled by having case data, hospital (& ICU data) and population by county, but those data were not available to the best of our knowledge.

In [9]:
#TODO: Turn this into a function so enddate can be passed
#TODO: Then build an animation of Days until ICU is filled -- this shows the robustness of the model.
#TODO: Update this daily, and figure out if we can easily publish this to the web and update a website daily.
#TODO: Add the ability now to plot the growth curve of states like https://colab.research.google.com/github/machine-learning-apps/covid19-dashboard/blob/master/_notebooks/2020-03-14-covid19_growth.ipynb


startdate = datetime.date(2020,3,10)
enddate = datetime.date(2020,3,18)
days = (enddate - startdate).days + 1

base_data=us_timeseries.iloc[:,-days:].sum()
print(base_data)
results=scipy.optimize.curve_fit(lambda day,gr: base_data[0]*pow(gr,day),  range(len(base_data)),  base_data,  p0=(2))
base_GR=results[0][0]
print("Base GR {:4.2f}".format(base_GR))

GROWTH_RATE_MIN_THRESHOLD = 100

# We define this as a function in case others would like to reuse this function to create their own dataframes for analysis
# Returns 2 datasets: one is a time series of counts by state for every day, and the second is the resultant analysis of these time series data
def create_dfs(case_concentration=1.0, population_concentration=1.0):
  us_plotseries = []
  us_growthseries =[]
  print("State,GR, ratio, Days to fill ICU")
  for index, state_row in us_timeseries.iterrows():
    state_name = state_row["Province/State"]
    if state_name not in us_state_abbrev or state_name not in us_state_pop:
      continue
    state_abbrev = us_state_abbrev[state_name]
    last_count = 0
    growth_rate = 0.0
    valid_data=[]
    for date in (startdate + datetime.timedelta(n) for n in range(days)):
      datestr = "" + str(date.month) + "/" + str(date.day) + "/" + str(date.year)[2:]
      count = state_row[datestr]
      row = [state_abbrev, str(date), count]
      if date.day == 9:
        print(row)
      us_plotseries.append(row)
      valid_data.append(count)
    ## Remove all data prior to 3/10 otherwise we end up with 0's overpowering the curves
    valid_data = valid_data[-days:]
    fit=scipy.optimize.curve_fit(lambda day,gr: valid_data[0]*pow(gr,day),  range(len(valid_data)),  valid_data,  p0=(2))
    GR_raw=fit[0][0]
    GR=GR_raw if valid_data[-1]>GROWTH_RATE_MIN_THRESHOLD else base_GR
    growth_rate=(GR-1)*100
    
    doubling_days = 10
    if GR > 1:
      doubling_days = np.log(2)/np.log(GR)
    
    population = us_state_pop[state_name]
    percent_population = count * 10000.0 / population

    beds = hospital_bed_count_df.loc[state_abbrev]
    beds_in_metro = beds * population_concentration
    cases_needing_beds_in_metro = count * case_concentration * .4
    percent_beds = cases_needing_beds_in_metro * 100.0 / (beds_in_metro)

    icu_beds = beds * 0.15
    cases_needing_icu_in_metro = count * case_concentration * .15
    icu_beds_in_metro = icu_beds * population_concentration
    percent_icu = cases_needing_icu_in_metro * 100.0 / (icu_beds_in_metro)

# Now let's compute the number of days until the ICU is at 100%

    multiplier_to_fill_icu = 1.0 / (percent_icu/100)
    days_to_fill=100
    if GR > 1.0:
      days_to_fill = math.log10(multiplier_to_fill_icu)/math.log10(GR)
      #print ("{:2},  {:3.1f}, {:5.0f}, {:5.0f}".format(state_abbrev,GR,multiplier_to_fill_icu,days_to_fill) )

    beds_per_capita = 1000.0 * beds / population
    growth_row = [state_abbrev, str(date), count, growth_rate, doubling_days, population, percent_population, beds, percent_beds, icu_beds, beds_per_capita, percent_icu, days_to_fill,GR]
    us_growthseries.append(growth_row)

  us_plotseries_df = pd.DataFrame(us_plotseries, columns =['State', 'Day','Count'], dtype = int) 
  us_growthseries_df = pd.DataFrame(us_growthseries, columns =['State', 'Day','Count', 'Growth Rate', 'Days to Double', 'Population', 'Population Percent (1/100 of 1%)', 'Beds', 'Beds Percent', 'ICU', 'Beds per 1000', 'ICU Percent', 'Days To ICU Capacity','GR'], dtype = int) 

  return us_plotseries_df, us_growthseries_df

us_plotseries_df, us_growthseries_df = create_dfs(case_concentration=0.9, population_concentration=0.4)
print("us_plot series",us_plotseries_df)
print (us_growthseries_df.head(20))


3/10/20     959
3/11/20    1281
3/12/20    1663
3/13/20    2179
3/14/20    2727
3/15/20    3499
3/16/20    4632
3/17/20    6421
3/18/20    7783
dtype: int64
Base GR 1.30
State,GR, ratio, Days to fill ICU
us_plot series     State         Day  Count
0      WA  2020-03-10    267
1      WA  2020-03-11    366
2      WA  2020-03-12    442
3      WA  2020-03-13    568
4      WA  2020-03-14    572
..    ...         ...    ...
463    PR  2020-03-14      3
464    PR  2020-03-15      5
465    PR  2020-03-16      5
466    PR  2020-03-17      5
467    PR  2020-03-18      5

[468 rows x 3 columns]
   State         Day  Count  ...  ICU Percent  Days To ICU Capacity        GR
0     WA  2020-03-18   1014  ...    20.204570              8.701950  1.201754
1     NY  2020-03-18   2495  ...    10.760907              6.838167  1.385417
2     CA  2020-03-18    751  ...     1.922005             18.377751  1.239901
3     MA  2020-03-18    218  ...     3.234208             29.858749  1.121784
4     GA  2020-03-1

/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning:

Covariance of the parameters could not be estimated



In [10]:
colorscale = px.colors.sequential.Blues_r
reverse_colorscale = colorscale [::-1]
fig = px.choropleth(us_plotseries_df, title = "Cases", locations="State", locationmode="USA-states",scope="usa", color="Count", animation_frame="Day", range_color=[0,250], color_continuous_scale=colorscale)
fig.show()



In [11]:
fig2 = px.choropleth(us_growthseries_df, title="Growth Rate", locations="State", locationmode="USA-states",scope="usa", color="Growth Rate", range_color=[0,50], color_continuous_scale=colorscale)
fig2.show()


In [12]:
fig3 = px.choropleth(us_growthseries_df, title="Days to Double", locations="State", locationmode="USA-states",scope="usa", color="Days to Double", range_color=[0,10], color_continuous_scale=reverse_colorscale)
fig3.show()


In [13]:
fig4 = px.choropleth(us_growthseries_df, title="Percent Infected (bps)", locations="State", locationmode="USA-states",scope="usa", color="Population Percent (1/100 of 1%)", range_color=[0,1], color_continuous_scale=colorscale)
fig4.show()


In [14]:
fig4a = px.choropleth(us_growthseries_df, title="Hospital Beds Per Capita (1000 people)", locations="State", locationmode="USA-states",scope="usa", color="Beds per 1000", range_color=[0,10], color_continuous_scale=colorscale)
fig4a.show()


In [15]:
fig5 = px.choropleth(us_growthseries_df, title="Percent of Beds (%)", locations="State", locationmode="USA-states",scope="usa", color="Beds Percent", range_color=[0,50], color_continuous_scale=colorscale)
fig5.show()


In [16]:
fig6 = px.choropleth(us_growthseries_df, title="Percent of ICU Beds (%)", locations="State", locationmode="USA-states",scope="usa", color="ICU Percent", range_color=[0,50], color_continuous_scale=colorscale)
fig6.show()


In [17]:
fig7 = px.choropleth(us_growthseries_df, title="Days to fill ICU", locations="State", locationmode="USA-states",scope="usa", color="Days To ICU Capacity", range_color=[0,50], color_continuous_scale=colorscale)
fig7.show()

```
DaysInICU=14 days in ICU
needICU = 15% of infected
infected after DaysInICU days= infected* GR**DaysInICU
newly infected= infected*GR**DaysInICU - infected
newly need ICU = newly_infected*needICU
infected=ICUBeds/needICU
ICUBeds=(infected)*(GR**DaysInICU-1)*needICU
ICUBeds=(ICUBeds/needICU)*(GR**DaysInICU-1)*needICU
1=GR**DaysInICU-1
GR=exp(log(2)/DaysInICU)
```


In [18]:
days_in_ICU=14
# Critical GrowthRate (GR) is the steady state infection growth rate to keep the ICU load constant
# it is independent of state o
print("Critical growth rate = {:.2f}%".format(100*(np.exp(np.log(2)/days_in_ICU)-1))
)

Critical growth rate = 5.08%


Next, let's compute a simulation to determine what the GrowthRate (GR) we need for this state to NOT overwhelm the ICU.
This simulation will have to be done including  assumed length of stay in the ICU (14 days for now--can correct if we need to) (people in then people out).


*   What does GR need to be tomorrow to get the number of days to 60?
*   What if we double the number ICU-capable beds?
*   Ensure we keep the # of days in the ICU as a variable.


In [19]:
us_growthseries_df.sort_values('Days To ICU Capacity')[['State', 'Count','Growth Rate', 'Days To ICU Capacity']]

,State,Count,Growth Rate,Days To ICU Capacity
34,LA,257,103.709493,4.557036
1,NY,2495,38.541696,6.838167
0,WA,1014,20.175377,8.701950
5,CO,184,40.006626,8.950870
7,NJ,267,46.017054,9.845408
6,FL,314,46.541070,11.592368
36,VT,18,30.259751,12.266534
4,GA,199,36.566550,12.784267
10,IL,162,41.289666,12.814838
49,WY,15,30.259751,13.467239


Compute how effective Italian measures were in impacting growth rates

In [20]:
italy_timeseries=global_timeseries.loc[['Italy']]
#italy_timeseries.loc[:,'3/10/20':'3/15/20']
italy_timeseries


,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Italy,NaN,43.0,12.0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,20,62,155,229,322,453,655,888,1128,1694,2036,2502,3089,3858,4636,5883,7375,9172,10149,12462,12462,17660,21157,24747,27980,31506,35713


In [22]:
#TODO: Add fit-by-date

growth_rates = []
last_count = 0
dnddate = datetime.date(2020,3,18)
startdate = datetime.date(2020,2,10)
days = (enddate - startdate).days + 1
for date in (startdate + datetime.timedelta(n) for n in range(days)):
  datestr = "" + str(date.month) + "/" + str(date.day) + "/" + str(date.year)[2:]
  count = italy_timeseries.get(datestr)
  italy_growth = 0.0
  if last_count > 0:
    italy_growth = 1.0 * int(count) /last_count - 1
  
  last_count = int(count)
  growth_rates.append([date, int(count), italy_growth])
growth_rates
italy_df = pd.DataFrame(data=growth_rates, columns=['Date', 'Count', 'Growth Rate'])
fig2 = px.line(data_frame=italy_df, x= 'Date', y='Count', title="Italian Count")
fig2.show()
fig3 = px.line(data_frame=italy_df, x= 'Date', y='Growth Rate', title="Italian Growth Rate")
fig3.update_yaxes(range=[0, 1])
fig3.show()